In [1]:
from selenium import webdriver
from pandas.io.html import read_html
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import numpy as np
import os
from bs4 import BeautifulSoup

In [68]:
def FB_match(url, sleep_time):
    
    driver = webdriver.Chrome()
    driver.get(url)
    
    time.sleep(sleep_time)
    
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    
    # 리스트 미리 만들어 놓기
    player_id_list = []
    player_name_list = []
    player_rating_list = []
    player_goal_list = []
    player_goal_sum_list = []
    player_as_list = []
    player_as_sum_list = []
    player_shots_list = []
    player_sot_list = []
    player_pass_list = []
    player_pa_list = []
    player_db_list = []
    player_dbw_list = []
    player_tk_list = []
    player_tks_list = []
    player_yc_list = []
    player_yc_sum_list = []
    player_rc_list = []
    player_rc_sum_list = []
    
    # 선수 id
    player_id = soup.find_all('div', class_ = "player")
    for id in player_id:
        player_id_list.append(id['data-player-id'])
    
    # 선수 이름
    player_name = soup.find_all('div', class_ = "player-name-wrapper")
    try:
        for name in player_name:
            player_name_list.append(name['title'])
    except:
        print("넘김")
    
    # 선수 평점
    player_rating = soup.find_all('span', class_ = "player-stat-value")
    for rating in player_rating:
        if rating.text == '':
            player_rating_list.append("None")
        else: player_rating_list.append(rating.text)
            
    # 선수 GOAL
    player_goal = soup.find_all('div', {"class": "incident-icon", "data-type":"16"})
    for goal in player_goal:
        player_goal_sum_list.append(goal['data-player-id'])
    player_goal_sum_list = player_goal_sum_list[:int(len(player_goal_sum_list)/3)]
    
    for id in player_id_list:
        goal_summary = 0
        for goal_id in player_goal_sum_list:
            if id == goal_id:
                goal_summary += 1
        if goal_summary != 0:
            player_goal_list.append(goal_summary)
        else : player_goal_list.append(0)
    
    # 선수 ASSIST
    player_assist = soup.find_all('div', {"class": "incident-icon", "data-type":["1", "61"]})
    for assist in player_assist:
        player_as_sum_list.append(assist['data-player-id'])
    player_as_sum_list = player_as_sum_list[:int(len(player_as_sum_list)/3)]
    
    for id in player_id_list:
        as_summary = 0
        for as_id in player_as_sum_list:
            if id == as_id:
                as_summary += 1
        if as_summary != 0:
            player_as_list.append(as_summary)
        else : player_as_list.append(0)
    
    # 선수 옐로카드
    player_yc = soup.find_all('div', {"class": "incident-icon", "data-card-type": ["31", "32"]})
    for yc in player_yc:
        player_yc_sum_list.append(yc['data-player-id'])
    player_yc_sum_list = player_yc_sum_list[:int(len(player_yc_sum_list)/3)]
    
    for id in player_id_list:
        yc_summary = 0
        for yc_id in player_yc_sum_list:
            if id == yc_id:
                yc_summary += 1
        if yc_summary != 0:
            player_yc_list.append(yc_summary)
        else : player_yc_list.append(0)
            
    # 선수 레드카드
    player_rc = soup.find_all('div', {"class": "incident-icon", "data-card-type": "33"})
    if len(player_rc) == 0: # 레드카드가 없을수도 있잖아.
        for id in player_id_list:
            player_rc_list.append(0)
    else:
        for rc in player_rc:
            player_rc_sum_list.append(rc['data-player-id'])
            player_rc_sum_list = player_rc_sum_list[:int(len(player_rc_sum_list)/3)]
        for id in player_id_list:
            for rc_id in player_rc_sum_list:
                if id == rc_id:
                    player_rc_list.append(1)
                else : player_rc_list.append(0)
    
    # 선수 당 슛 횟수
    driver.execute_script("window.scrollTo(0, 1500)")
    time.sleep(sleep_time)
    accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[1]/li[2]/div[2]/label')
    accept.click()
    time.sleep(sleep_time)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    player_shots = soup.find_all('span', class_ = "player-stat-value")
    
    for shots in player_shots:
        if shots.text == "":
            player_shots_list.append("0")
        else : player_shots_list.append(shots.text)
            
    # SOT(shots on target) : 유효 슈팅
    accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[2]/li[2]/div[1]/ul/li[3]')
    accept.click()
    time.sleep(sleep_time)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    player_sot = soup.find_all('span', class_ = "player-stat-value")
    
    for sot in player_sot:
        if sot.text == "":
            player_sot_list.append("0")
        else : player_sot_list.append(sot.text)
            
    # Pass
    accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[1]/li[4]/div[2]/label')
    accept.click()
    time.sleep(sleep_time)
    accept.click()
    time.sleep(sleep_time)
    accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[2]/li[4]/div[1]/ul/li[2]')
    accept.click()
    time.sleep(sleep_time)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    player_pass = soup.find_all('span', class_ = "player-stat-value")
    
    for passes in player_pass:
        if passes.text == "":
            player_pass_list.append("0")
        else: player_pass_list.append(passes.text)
            
    # PA : Accurate Passes
    time.sleep(sleep_time)
    accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[2]/li[4]/div[1]/ul/li[3]')
    accept.click()
    time.sleep(sleep_time)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    player_pa = soup.find_all('span', class_ = "player-stat-value")
    
    for pa in player_pa:
        if pa.text == "":
            player_pa_list.append("0")
        else: player_pa_list.append(pa.text)
    
    # Dribbles
    accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[1]/li[5]/div[2]/label')
    accept.click()
    time.sleep(sleep_time)
    accept.click()
    time.sleep(sleep_time)
    accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[2]/li[5]/div[1]/ul/li[2]')
    accept.click()
    time.sleep(sleep_time)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    player_db = soup.find_all('span', class_ = "player-stat-value")
    
    for db in player_db:
        if db.text == "":
            player_db_list.append("0")
        else: player_db_list.append(db.text)
            
    # DB_W : Dribbles Won
    accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[2]/li[5]/div[1]/ul/li[1]')
    accept.click()
    time.sleep(sleep_time)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    player_dbw = soup.find_all('span', class_ = "player-stat-value")
    
    for dbw in player_dbw:
        if dbw.text == "":
            player_dbw_list.append("0")
        else: player_dbw_list.append(dbw.text)
            
    # TK : Tackles
    accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[1]/li[7]/div[2]/label')
    accept.click()
    time.sleep(sleep_time)
    accept.click()
    time.sleep(sleep_time)
    accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[2]/li[7]/div[1]/ul/li[2]')
    accept.click()
    time.sleep(sleep_time)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    player_tk = soup.find_all('span', class_ = "player-stat-value")
    
    for tk in player_tk:
        if tk.text == "":
            player_tk_list.append("0")
        else: player_tk_list.append(tk.text)
    
    # TK_S : Successful Tackles 
    accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[2]/li[7]/div[1]/ul/li[1]')
    accept.click()
    time.sleep(sleep_time)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")
    player_tks = soup.find_all('span', class_ = "player-stat-value")
    
    for tks in player_tks:
        if tks.text == "":
            player_tks_list.append("0")
        else: player_tks_list.append(tks.text)
    
    # 결과 df
    df = pd.DataFrame({
        "ID" : player_id_list,
        "Player" : player_name_list,
        "Rating" : player_rating_list,
        "Goal" : player_goal_list,
        "Assist" : player_as_list,
        "Shots" : player_shots_list,
        "SOT" : player_sot_list,
        "Pass" : player_pass_list,
        "PA" : player_pa_list,
        "DB" : player_db_list,
        "DB_W" : player_dbw_list,
        "TK" : player_tk_list,
        "TK_S" : player_tks_list,
        "YC" : player_yc_list,
        "RC" : player_rc_list,
    })
    return df

In [69]:
url = 'https://1xbet.whoscored.com/Matches/1549703/Live/England-Premier-League-2021-2022-Leicester-Tottenham'
df = FB_match(url, 2)
df

넘김


C:\Users\KANGSA~1\AppData\Local\Temp/ipykernel_11244/3336160233.py:115: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[1]/li[2]/div[2]/label')
C:\Users\KANGSA~1\AppData\Local\Temp/ipykernel_11244/3336160233.py:128: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[2]/li[2]/div[1]/ul/li[3]')
C:\Users\KANGSA~1\AppData\Local\Temp/ipykernel_11244/3336160233.py:141: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  accept = driver.find_element_by_xpath('//*[@id="match-centre-stats"]/div[1]/ul[1]/li[4]/div[2]/label')
C:\Users\KANGSA~1\AppData\Local\Temp/ipykernel_11244/3336160233.py:146: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  acce

,ID,Player,Rating,Goal,Assist,Shots,SOT,Pass,PA,DB,DB_W,TK,TK_S,YC,RC
0,301441,James Justin,6.2,0,0,0,0,23,20,1,0,0,0,0,0
1,331382,Harvey Barnes,6.7,0,1,1,0,5,5,1,1,0,0,0,0
2,345664,Boubakary Soumaré,5.9,0,0,0,0,4,4,0,0,0,0,0,0
3,135366,Ayoze Pérez,None,0,0,0,0,0,0,0,0,0,0,0,0
4,18310,Eldin Jakupovic,None,0,0,0,0,0,0,0,0,0,0,0,0
5,429531,Kasey McAteer,None,0,0,0,0,0,0,0,0,0,0,0,0
6,429681,Lewis Brunt,None,0,0,0,0,0,0,0,0,0,0,0,0
7,99165,Danny Ward,None,0,0,0,0,0,0,0,0,0,0,0,0
8,409879,Vontae Daley-Campbell,None,0,0,0,0,0,0,0,0,0,0,0,0
9,19545,Kasper Schmeichel,6.6,0,0,0,0,37,25,0,0,2,0,0,0
